In [49]:
# https://ta-lib.github.io/ta-lib-python/index.html
from talib import abstract
import pandas as pd
import numpy as np
import talib
from talib.abstract import *
from quantfreedom._typing import (
    plFrame, plSeries, Union, pdFrame, pdSeries,
)
df = pd.read_csv('30min.csv', index_col='time')

In [71]:
def testing(
    func_name: str,
    df_prices: pdFrame,
    cart_product: bool,
    combos: bool,
    **kwargs,
):
    users_args_list = []
    biggest = 1
    info = Function(func_name).info
    input_names_dict = {}
    in_names_key = ''
    output_names = info['output_names']

    for in_names_key, in_names_values in info['input_names'].items():
        filled = False
        if isinstance(in_names_values, list):
            for kwarg_keys, kwarg_values in kwargs.items():
                if in_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, list):
                        raise ValueError(
                            f'{in_names_key} must be a list of strings')

                    if isinstance(kwarg_values, list):
                        if not all(isinstance(x, str) for x in kwarg_values):
                            raise ValueError(
                                f'{in_names_key} your list has to be made up of strings')

                        if len(kwarg_values) == 1:
                            raise ValueError(
                                f'{in_names_key} your list length must be greater than 1')

                        if len(kwarg_values) != len(in_names_values):
                            raise ValueError(
                                f'{in_names_key} your list length must be {len(in_names_values)}')

                    input_names_dict[in_names_key] = kwarg_values
                    filled = True
                    break

            if not filled:
                input_names_dict[in_names_key] = in_names_values

        elif isinstance(in_names_values, str):
            for kwarg_keys, kwarg_values in kwargs.items():
                if in_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, str):
                        raise ValueError(
                            f'{in_names_key} must be a string and not a list of strings')

                    input_names_dict[in_names_key] = kwarg_values
                    filled = True
                    break
                elif in_names_key == 'price' and kwarg_keys == 'prices':
                    raise ValueError(
                            f'you need to provide price instead of prices')
                elif in_names_key == 'prices' and kwarg_keys == 'price':
                    raise ValueError(
                            f'you need to provide prices instead of price')

            if filled:
                input_names_dict[in_names_key] = in_names_values

    if bool(info['parameters']):
        param_dict = {}
        for param_names_key, param_names_values in info['parameters'].items():
            filled = False
            for kwarg_keys, kwarg_values in kwargs.items():
                if param_names_key == kwarg_keys:
                    if not isinstance(kwarg_values, (int, float, list)):
                        raise ValueError(
                            f'{param_names_key} must be a int float or a list of ints or floats')

                    if isinstance(kwarg_values, (int, float)):
                        users_args_list.append(np.asarray([kwarg_values]))

                    elif isinstance(kwarg_values, list):
                        if cart_product == False and combos == False and len(info['parameters']) > 1:
                            raise ValueError(
                                f'{param_names_key} you can\'t have list items as args with indicators that have mutiple params without doing a combo or cart product')
                        if not all(isinstance(x, (int, float)) for x in kwarg_values):
                            raise ValueError(
                                f'{param_names_key} your list has to be filled with ints or floats')
                        if len(kwarg_values) == 1:
                            raise ValueError(
                                f'{param_names_key} your list length must be greater than 1')
                        if combos:
                            if biggest == 1 and len(kwarg_values) > 1:
                                biggest = len(kwarg_values)
                            if biggest != len(kwarg_values):
                                raise ValueError(
                                    f'{param_names_key} when using combos all listed items must be same length')

                        users_args_list.append(np.asarray(kwarg_values))
                    param_dict[param_names_key] = kwarg_values
                    filled = True
                    break

            if not filled:
                param_dict[param_names_key] = param_names_values
                users_args_list.append(np.asarray([param_names_values]))
    else:
        param_dict = ()

    if combos and biggest != 1:
        final_tuple = []
        for x in users_args_list:
            if x.size == 1:
                final_tuple.append(np.broadcast_to(x, biggest))
            else:
                final_tuple.append(x)
        final_tuple = tuple(final_tuple)

    elif cart_product and biggest != 1:
        users_args_tuple = tuple(users_args_list)
        n = 1
        for x in users_args_tuple:
            n *= x.size
        final_tuple = np.empty((n, len(users_args_tuple)))

        for i in range(len(users_args_tuple)):
            m = int(n / users_args_tuple[i].size)
            final_tuple[:n, i] = np.repeat(users_args_tuple[i], m)
            n //= users_args_tuple[i].size

        n = users_args_tuple[-1].size
        for k in range(len(users_args_tuple)-2, -1, -1):
            n *= users_args_tuple[k].size
            m = int(n / users_args_tuple[k].size)
            for j in range(1, users_args_tuple[k].size):
                final_tuple[j*m:(j+1)*m, k+1:] = final_tuple[0:m, k+1:]
                final_tuple = final_tuple.T
                
    elif biggest == 1 and (cart_product or combos):
        raise ValueError(f'You have to have a list for paramaters {list(info["parameters"].keys())} to use cart product or combos')
    else:
        final_tuple = tuple(users_args_list)

    ind_settings_tup = ()
    ind_setings_len = final_tuple[0].size
    final_df_dict = {}

    for o_names in output_names:
        final_df_dict[o_names] = pd.DataFrame()

    # c is the indicator result of the array within the tuple (array[x], array[x])
    for c in range(ind_setings_len):
        # x is the array object in the tuple (x,x)
        for x in final_tuple:
            if type(x[c]) == np.int32:
                ind_settings_tup = ind_settings_tup + (int(x[c]),)
            if type(x[c]) == np.float32:
                ind_settings_tup = ind_settings_tup + (float(x[c]),)
        
        if in_names_key == 'price':
            # these are the names called by the fun like talib('rsi').real - real is the output name
            for out_name in output_names:
                # setting temp df of result out name
                temp_df = Function(func_name)(
                    df_prices, *
                    ind_settings_tup, price=input_names_dict[in_names_key]
                )[out_name]

                # creating multindex for new results
                param_keys = list(info['parameters'].keys())
                param_keys.insert(0, out_name + '_count')
                temp_df.columns = pd.MultiIndex.from_tuples(
                    [(c,) + ind_settings_tup],
                    names=param_keys
                )

                # getting df from within list and concating to it
                final_df_dict[out_name] = pd.concat(
                    [final_df_dict[out_name], temp_df], axis=1)
        
        elif in_names_key == 'prices':
            # these are the names called by the fun like talib('rsi').real - real is the output name
            for out_name in output_names:
                # setting temp df of result out name
                temp_df = Function(func_name)(
                    df_prices, *
                    ind_settings_tup, prices=input_names_dict[in_names_key]
                )[out_name].to_frame()

                # creating multindex for new results
                param_keys = list(info['parameters'].keys())
                param_keys.insert(0, out_name + '_count')
                temp_df.columns = pd.MultiIndex.from_tuples(
                    tuples=[(c,) + ind_settings_tup],
                    names=param_keys
                )

                # getting df from within list and concating to it
                final_df_dict[out_name] = pd.concat(
                    [final_df_dict[out_name], temp_df], axis=1)

        ind_settings_tup = ()
    return final_df_dict


dfs = testing(
    func_name='rsi',
    fastk_period=[6, 7],
    slowk_period=2,
    slowd_period=3,
    price='open',
    cart_product=False,
    combos=False,
    df_prices=df,
)
dfs['real']

KeyError: 'real'

In [3]:
(3,4,5,6) + (1,)

(3, 4, 5, 6, 1)

In [4]:
dfs['slowd']

slowd,0,1
fastk_period,6,3
slowk_period,2,2
slowk_matype,0,0
slowd_period,3,3
slowd_matype,0,0
time,,
2020-10-01 00:00:00,NaN,NaN
2020-10-01 00:30:00,NaN,NaN
2020-10-01 01:00:00,NaN,NaN
2020-10-01 01:30:00,NaN,NaN


In [5]:
dfs[0]

KeyError: 0

In [ ]:
dfs[0]

In [55]:
Function('AVGPRICE').info

{'name': 'AVGPRICE',
 'group': 'Price Transform',
 'display_name': 'Average Price',
 'function_flags': ['Output scale same as input'],
 'input_names': OrderedDict([('prices', ['open', 'high', 'low', 'close'])]),
 'parameters': OrderedDict(),
 'output_flags': OrderedDict([('real', ['Line'])]),
 'output_names': ['real']}

In [14]:
Function('stoch').info

{'name': 'STOCH',
 'group': 'Momentum Indicators',
 'display_name': 'Stochastic',
 'function_flags': None,
 'input_names': OrderedDict([('prices', ['high', 'low', 'close'])]),
 'parameters': OrderedDict([('fastk_period', 5),
              ('slowk_period', 3),
              ('slowk_matype', 0),
              ('slowd_period', 3),
              ('slowd_matype', 0)]),
 'output_flags': OrderedDict([('slowk', ['Dashed Line']),
              ('slowd', ['Dashed Line'])]),
 'output_names': ['slowk', 'slowd']}

In [ ]:
hey = ['hey', 'high', 'yes']
hey.insert(0, 'meme')
hey

In [ ]:
temp = (3,) + (2, 3, 5)
temp

In [ ]:
test = Function('stoch')(df, *(9,1,5))['slowk'].to_frame()
test.columns = pd.MultiIndex.from_tuples([temp], names=hey)
test

In [ ]:
test2 = pd.DataFrame(Function('stoch')(df, *(5, 2, 8))['slowd'])
test2.columns = pd.MultiIndex.from_tuples(
    [(30, 64, 94)], names=['hey', 'high', 'yes'])
test2


In [ ]:
pd.concat([test, test2], axis=1)

In [ ]:
list_list = [pd.DataFrame(), pd.DataFrame()]
empty = pd.DataFrame()
list_list[0] = pd.concat([list_list[0], test])
list_list[0]

In [ ]:
pd.MultiIndex.from_arrays([[2,3], ['hey', 'high']], names=('hey', 'high'))

In [64]:
talib.STOCH(df.close, df.open, df.open).to_frame()

AttributeError: 'tuple' object has no attribute 'to_frame'

In [56]:
Function('AVGPRICE')(df).to_frame()

,0
time,
2020-10-01 00:00:00,10781.375
2020-10-01 00:30:00,10783.125
2020-10-01 01:00:00,10785.000
2020-10-01 01:30:00,10810.875
2020-10-01 02:00:00,10820.000
...,...
2021-02-22 21:30:00,54542.000
2021-02-22 22:00:00,54592.875
2021-02-22 22:30:00,54119.625


In [70]:
Function('stoch')(df)['slowk']

time
2020-10-01 00:00:00          NaN
2020-10-01 00:30:00          NaN
2020-10-01 01:00:00          NaN
2020-10-01 01:30:00          NaN
2020-10-01 02:00:00          NaN
                         ...    
2021-02-22 21:30:00    88.730291
2021-02-22 22:00:00    85.454038
2021-02-22 22:30:00    63.341889
2021-02-22 23:00:00    35.650502
2021-02-22 23:30:00    31.639017
Name: slowk, Length: 6960, dtype: float64

In [69]:
Function('rsi')(df)['real']

KeyError: 'real'